In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys

sys.path.append("../")

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import DPTForDepthEstimation

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

from src.models.depth_encoder import DepthEncoder